In [1]:
#Bu Dosyada şu ana kadar ayrı ayrı test ettiğim  HOG + LBP + Gabor özniteliklerini birleştireceğim
#PCA uygulayacağım boyutu düşürüceğim
#Tüm modelleri bu yeni öznitelik setiyle eğiteceğim ve en yüksek doğruluk oranlarıma ulaşmaya çalışacağım

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report


In [3]:
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor

# Veri klasörünün yolu
data_directory = "Data_2"

# Sınıfların listesini al (.DS_Store gibi gizli dosyaları filtrele önceki train dosyalarında bahsettiğim gibi MAC kullandığım için oluşan gizli bir klasör)
classes = sorted([
    d for d in os.listdir(data_directory)
    if os.path.isdir(os.path.join(data_directory, d)) and not d.startswith(".")
])


In [4]:
print("Sınıflar:", classes)

Sınıflar: ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [5]:
images = []
labels = []

# Görüntüleri oku ve etiketle
for cls in classes:
    class_folder = os.path.join(data_directory, cls)
    for file in os.listdir(class_folder):
        img_path = os.path.join(class_folder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Geçersiz resimleri atla
        if img is None:
            continue

        img = cv2.resize(img, (64, 64))
        images.append(img)
        labels.append(cls)

# Numpy dizilerine çevir
images = np.array(images, dtype=np.uint8) #float uyarı verebiliyordu
labels = LabelEncoder().fit_transform(labels)

In [6]:
print("Toplam görüntü sayısı:", len(images))
print("Sınıf sayısı:", len(np.unique(labels)))

Toplam görüntü sayısı: 17460
Sınıf sayısı: 5


In [7]:
hog_features = []

for img in images:
    # HOG öznitelik çıkarımı
    feat = hog(
        img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        visualize=False,
        feature_vector=True
    )
    hog_features.append(feat)

hog_features = np.array(hog_features, dtype=np.float32)
print("HOG öznitelik boyutu:", hog_features.shape)


HOG öznitelik boyutu: (17460, 1764)


In [8]:
# LBP (Local Binary Pattern) parametreleri:
radius = 1                           # Merkez pikselin çevresindeki halka yarıçapı
n_points = 8 * radius                # Komşu piksel sayısı (8 komşu)
#Literatür taramasında bulduğum değerleri kullandım https://ieeexplore.ieee.org/document/1576816
#Sayfa 2, ilk paragraf:
#"The original LBP operator labels the pixels of an image by thresholding the 3x3-neighborhood of each pixel with the center pixel value."
#Bu 3x3 neighborhood demek: radius=1 ve P=8 anlamına gelir.

lbp_features = []                   

for img in images:
    # LBP hesapla (uniform yöntemi: geçiş sayısına göre histogramda sadeleştirme yapar)
    lbp = local_binary_pattern(img, P=n_points, R=radius, method="uniform")

    # LBP değerleri üzerinden histogram oluştur
    hist, _ = np.histogram(
        lbp.ravel(),                            # 1D diziye çevir (histogram için)
        bins=np.arange(0, n_points + 3),        # Bin sayısı: P+2 (uniform için)
        range=(0, n_points + 2)                 # Aralık: [0, P+2]
    )

    
    hist = hist.astype("float32")               # Histogram değerlerini float'a çevir 
    hist /= (hist.sum() + 1e-6)                 # Sıfıra bölünmeyi engellemek için küçük sabit

    
    lbp_features.append(hist)


lbp_features = np.array(lbp_features, dtype=np.float32)


print("LBP öznitelik boyutu:", lbp_features.shape)


LBP öznitelik boyutu: (17460, 10)


In [9]:
gabor_features = []  

# Farklı frekanslar – düşükten yükseğe (detay seviyesini kontrol eder)
frequencies = [0.1, 0.3, 0.5]

# 4 farklı yön (0°, 45°, 90°, 135°) – yönelime duyarlı doku bilgisi sağlar
thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]

for img in images:
    feats = []  # 
    for theta in thetas:  # Her yön için
        for freq in frequencies:  # Her frekans için
            filt_real, _ = gabor(img, frequency=freq, theta=theta)  # Gabor filtresi uygula
            feats.append(filt_real.mean())  # Ortalama değer – doku yoğunluğu
            feats.append(filt_real.var())   # Varyans – doku çeşitliliği
    gabor_features.append(feats)


gabor_features = np.array(gabor_features)
print("Gabor öznitelik boyutu:", gabor_features.shape)


Gabor öznitelik boyutu: (17460, 24)


In [10]:
#Aşırı uzun sürüyor yukardaki kod
print("Gabor öznitelik boyutu:", gabor_features.shape)

Gabor öznitelik boyutu: (17460, 24)


In [11]:
#Çok uzun sürdüğü için kullanmama gerekirse diye .npy dosyalarına kaydediyorum
np.save("gabor_features.npy", gabor_features)
np.save("labels.npy", labels)  # Etiketleri de kaydet


In [12]:
# HOG, LBP ve Gabor özniteliklerini birleştir
X_combined = np.concatenate([hog_features, lbp_features, gabor_features], axis=1)
print("Birleşik öznitelik vektörü boyutu:", X_combined.shape)

# Not: axis=1 → her bir görüntü için öznitelikleri yatayda birleştirir (özellik bazında)
#     axis=0 olsaydı görüntüleri alt alta eklerdi (yanlış olurdu)

# Ölçekleme: Özellikleri standart normal dağılıma sokar (ortalama 0, std sapma 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)
print("Ölçekleme tamamlandı. İlk örnekten bir kesit:", X_scaled[0][:5])


Birleşik öznitelik vektörü boyutu: (17460, 1798)
Ölçekleme tamamlandı. İlk örnekten bir kesit: [-1.29794944 -1.06263768 -0.49759818 -0.5498964   2.10344826]


In [13]:
# PCA 
# n_components=0.95: Verideki toplam bilginin (varyansın) %95’ini koruyacak şekilde bileşen sayısını otomatik seçer.
# Yani PCA, en fazla bilgiyi taşıyan bileşenleri seçecek ve geri kalanını atacak.
# Bu sayede hem boyut azaltılır hem de modelin aşırı öğrenmesi (overfitting) önlenebilir.
pca = PCA(n_components=0.95)

# PCA uygulanarak veriden (X_scaled) yeni bir boyuta indirgenmiş temsili olan (X_pca) elde ederiz.
X_pca = pca.fit_transform(X_scaled)

print("PCA sonrası boyut:", X_pca.shape)
print("Seçilen bileşen sayısı:", pca.n_components_)


PCA sonrası boyut: (17460, 500)
Seçilen bileşen sayısı: 500


In [14]:
# PCA ile boyutu azaltılmış veriyi eğitim ve test kümelerine ayırır
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, labels, test_size=0.2, random_state=42, stratify=labels
)


In [15]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score

# Özelleştirilmiş skorlayıcılar (zero_division=0 ile uyarı bastırılır)
skorlayicilar = {
    'accuracy': make_scorer(accuracy_score),
    'precision_macro': make_scorer(precision_score, average='macro', zero_division=0),
    'recall_macro': make_scorer(recall_score, average='macro', zero_division=0),
    'f1_macro': make_scorer(f1_score, average='macro', zero_division=0),
}

# SVM parametre kombinasyonları
svm_parametre_listesi = [
    {"C": 1, "gamma": "scale"},
    {"C": 10, "gamma": "scale"},
    {"C": 1, "gamma": 0.01},
    {"C": 10, "gamma": 0.01},
    {"C": 100, "gamma": 0.001},
    {"C": 0.1, "gamma": 1},
]

katlamali_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, ayar in enumerate(svm_parametre_listesi, start=1):
    print(f"\n=== SVM Model {index} | C: {ayar['C']}, gamma: {ayar['gamma']} ===")

    siniflandirici = SVC(C=ayar["C"], gamma=ayar["gamma"], kernel="rbf")

    sonuc = cross_validate(
        estimator=siniflandirici,
        X=X_pca,
        y=labels,
        cv=katlamali_cv,
        scoring=skorlayicilar,
        return_train_score=False,
        n_jobs=-1
    )

    print(f"Eğitim Süresi (ortalama): {sonuc['fit_time'].mean():.4f} saniye")
    print(f"Test Süresi (ortalama): {sonuc['score_time'].mean():.4f} saniye")

    for metrik in skorlayicilar.keys():
        skor = sonuc[f'test_{metrik}'].mean()
        print(f"{metrik.replace('_', ' ').title()} (macro): {skor:.4f}")



=== SVM Model 1 | C: 1, gamma: scale ===
Eğitim Süresi (ortalama): 46.0357 saniye
Test Süresi (ortalama): 8.8973 saniye
Accuracy (macro): 0.5905
Precision Macro (macro): 0.5975
Recall Macro (macro): 0.5656
F1 Macro (macro): 0.5752

=== SVM Model 2 | C: 10, gamma: scale ===
Eğitim Süresi (ortalama): 58.4740 saniye
Test Süresi (ortalama): 9.8829 saniye
Accuracy (macro): 0.6056
Precision Macro (macro): 0.6049
Recall Macro (macro): 0.5867
F1 Macro (macro): 0.5936

=== SVM Model 3 | C: 1, gamma: 0.01 ===
Eğitim Süresi (ortalama): 58.2657 saniye
Test Süresi (ortalama): 10.6784 saniye
Accuracy (macro): 0.3393
Precision Macro (macro): 0.8443
Recall Macro (macro): 0.2544
F1 Macro (macro): 0.1913

=== SVM Model 4 | C: 10, gamma: 0.01 ===
Eğitim Süresi (ortalama): 57.7769 saniye
Test Süresi (ortalama): 10.1571 saniye
Accuracy (macro): 0.3401
Precision Macro (macro): 0.8436
Recall Macro (macro): 0.2553
F1 Macro (macro): 0.1930

=== SVM Model 5 | C: 100, gamma: 0.001 ===
Eğitim Süresi (ortalama): 

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate

# MLP parametre kombinasyonları
mlp_parametre_listesi = [
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100, 50), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (200,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.01},
    {"hidden_layer_sizes": (100, 50, 25), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (150,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (256, 128), "learning_rate_init": 0.0005},
    {"hidden_layer_sizes": (128, 64, 32), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (64, 64, 64), "learning_rate_init": 0.005},
    {"hidden_layer_sizes": (512, 256, 128, 64), "learning_rate_init": 0.001},
]

# 5 katlı stratified cross-validation
katlamali_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Değerlendirilecek metrikler
performans_kriterleri = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Modellerin eğitim ve değerlendirme süreci
for index, ayar in enumerate(mlp_parametre_listesi, start=1):
    print(f"\n=== MLP Model {index} | Katmanlar: {ayar['hidden_layer_sizes']}, Öğrenme Oranı: {ayar['learning_rate_init']} ===")

    siniflandirici = MLPClassifier(
        hidden_layer_sizes=ayar["hidden_layer_sizes"],
        learning_rate_init=ayar["learning_rate_init"],
        activation='relu',
        max_iter=300,
        early_stopping=True,
        random_state=42
    )

    sonuc = cross_validate(
        estimator=siniflandirici,
        X=X_pca,
        y=labels,
        cv=katlamali_cv,
        scoring=performans_kriterleri,
        return_train_score=False,
        n_jobs=-1
    )

    # Süreleri yazdır
    print(f"Eğitim Süresi (ortalama): {sonuc['fit_time'].mean():.4f} saniye")
    print(f"Test Süresi (ortalama): {sonuc['score_time'].mean():.4f} saniye")

    # Metrikleri yazdır
    for metrik in performans_kriterleri:
        skor = sonuc[f'test_{metrik}'].mean()
        print(f"{metrik.replace('_', ' ').title()} (macro): {skor:.4f}")



=== MLP Model 1 | Katmanlar: (100,), Öğrenme Oranı: 0.001 ===
Eğitim Süresi (ortalama): 1.5041 saniye
Test Süresi (ortalama): 0.0061 saniye
Accuracy (macro): 0.5186
Precision Macro (macro): 0.5051
Recall Macro (macro): 0.5041
F1 Macro (macro): 0.5040

=== MLP Model 2 | Katmanlar: (100, 50), Öğrenme Oranı: 0.001 ===
Eğitim Süresi (ortalama): 1.1590 saniye
Test Süresi (ortalama): 0.0070 saniye
Accuracy (macro): 0.5109
Precision Macro (macro): 0.4990
Recall Macro (macro): 0.4958
F1 Macro (macro): 0.4959

=== MLP Model 3 | Katmanlar: (200,), Öğrenme Oranı: 0.001 ===
Eğitim Süresi (ortalama): 3.4686 saniye
Test Süresi (ortalama): 0.0113 saniye
Accuracy (macro): 0.5353
Precision Macro (macro): 0.5226
Recall Macro (macro): 0.5197
F1 Macro (macro): 0.5205

=== MLP Model 4 | Katmanlar: (100,), Öğrenme Oranı: 0.01 ===
Eğitim Süresi (ortalama): 0.9013 saniye
Test Süresi (ortalama): 0.0057 saniye
Accuracy (macro): 0.5098
Precision Macro (macro): 0.5009
Recall Macro (macro): 0.4950
F1 Macro (macro

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time

# Parametre kombinasyonları
rf_parametre_listesi = [
    {"n_estimators": 100, "max_depth": None},
    {"n_estimators": 200, "max_depth": None},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 10},
    {"n_estimators": 150, "max_depth": 20},
    {"n_estimators": 300, "max_depth": 15},
]

# 5 katlı stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tüm modeller için
for index, ayar in enumerate(rf_parametre_listesi, start=1):
    print(f"\n=== Random Forest Model {index} | n_estimators: {ayar['n_estimators']}, max_depth: {ayar['max_depth']} ===")

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    fit_times = []
    score_times = []

    for train_idx, test_idx in cv.split(X_pca, labels):
        X_train, X_test = X_pca[train_idx], X_pca[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        model = RandomForestClassifier(
            n_estimators=ayar["n_estimators"],
            max_depth=ayar["max_depth"],
            random_state=42,
            n_jobs=-1
        )

        start_fit = time.time()
        model.fit(X_train, y_train)
        end_fit = time.time()

        start_score = time.time()
        y_pred = model.predict(X_test)
        end_score = time.time()

        # Metrikler
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

        # Süreler
        fit_times.append(end_fit - start_fit)
        score_times.append(end_score - start_score)

    # Ortalamaları yaz
    print(f"Eğitim Süresi (ortalama): {np.mean(fit_times):.4f} saniye")
    print(f"Test Süresi (ortalama): {np.mean(score_times):.4f} saniye")
    print(f"Accuracy (macro): {np.mean(accuracy_scores):.4f}")
    print(f"Precision (macro): {np.mean(precision_scores):.4f}")
    print(f"Recall (macro): {np.mean(recall_scores):.4f}")
    print(f"F1-Score (macro): {np.mean(f1_scores):.4f}")



=== Random Forest Model 1 | n_estimators: 100, max_depth: None ===
Eğitim Süresi (ortalama): 4.0766 saniye
Test Süresi (ortalama): 0.0162 saniye
Accuracy (macro): 0.4381
Precision (macro): 0.5197
Recall (macro): 0.3752
F1-Score (macro): 0.3770

=== Random Forest Model 2 | n_estimators: 200, max_depth: None ===
Eğitim Süresi (ortalama): 7.4950 saniye
Test Süresi (ortalama): 0.0309 saniye
Accuracy (macro): 0.4463
Precision (macro): 0.5681
Recall (macro): 0.3797
F1-Score (macro): 0.3791

=== Random Forest Model 3 | n_estimators: 100, max_depth: 10 ===
Eğitim Süresi (ortalama): 2.3816 saniye
Test Süresi (ortalama): 0.0204 saniye
Accuracy (macro): 0.4040
Precision (macro): 0.6033
Recall (macro): 0.3263
F1-Score (macro): 0.3077

=== Random Forest Model 4 | n_estimators: 200, max_depth: 10 ===
Eğitim Süresi (ortalama): 4.6700 saniye
Test Süresi (ortalama): 0.0301 saniye
Accuracy (macro): 0.4080
Precision (macro): 0.6389
Recall (macro): 0.3283
F1-Score (macro): 0.3071

=== Random Forest Model

In [18]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time

# Parametre kombinasyonları
xgb_parametre_listesi = [
    {"learning_rate": 0.1, "max_depth": 3, "n_estimators": 100},
    {"learning_rate": 0.1, "max_depth": 5, "n_estimators": 100},
    {"learning_rate": 0.05, "max_depth": 4, "n_estimators": 150},
    {"learning_rate": 0.2, "max_depth": 3, "n_estimators": 100},
    {"learning_rate": 0.1, "max_depth": 6, "n_estimators": 200},
    {"learning_rate": 0.05, "max_depth": 5, "n_estimators": 300},
]

# 5 katlı stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Modelleri döngüyle eğitiş
for index, ayar in enumerate(xgb_parametre_listesi, start=1):
    print(f"\n=== XGBoost Model {index} | learning_rate: {ayar['learning_rate']}, max_depth: {ayar['max_depth']}, n_estimators: {ayar['n_estimators']} ===")

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    fit_times = []
    score_times = []

    for train_idx, test_idx in cv.split(X_pca, labels):
        X_train, X_test = X_pca[train_idx], X_pca[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        model = XGBClassifier(
            learning_rate=ayar["learning_rate"],
            max_depth=ayar["max_depth"],
            n_estimators=ayar["n_estimators"],
            use_label_encoder=False,
            eval_metric='mlogloss',
            verbosity=0,
            random_state=42,
            n_jobs=-1
        )

        start_fit = time.time()
        model.fit(X_train, y_train)
        end_fit = time.time()

        start_score = time.time()
        y_pred = model.predict(X_test)
        end_score = time.time()

        # Metrikler
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

        # Süreler
        fit_times.append(end_fit - start_fit)
        score_times.append(end_score - start_score)


    print(f"Eğitim Süresi (ortalama): {np.mean(fit_times):.4f} saniye")
    print(f"Test Süresi (ortalama): {np.mean(score_times):.4f} saniye")
    print(f"Accuracy (macro): {np.mean(accuracy_scores):.4f}")
    print(f"Precision (macro): {np.mean(precision_scores):.4f}")
    print(f"Recall (macro): {np.mean(recall_scores):.4f}")
    print(f"F1-Score (macro): {np.mean(f1_scores):.4f}")



=== XGBoost Model 1 | learning_rate: 0.1, max_depth: 3, n_estimators: 100 ===
Eğitim Süresi (ortalama): 2.7936 saniye
Test Süresi (ortalama): 0.0024 saniye
Accuracy (macro): 0.4987
Precision (macro): 0.5032
Recall (macro): 0.4585
F1-Score (macro): 0.4639

=== XGBoost Model 2 | learning_rate: 0.1, max_depth: 5, n_estimators: 100 ===
Eğitim Süresi (ortalama): 6.4823 saniye
Test Süresi (ortalama): 0.0041 saniye
Accuracy (macro): 0.5269
Precision (macro): 0.5340
Recall (macro): 0.4924
F1-Score (macro): 0.5004

=== XGBoost Model 3 | learning_rate: 0.05, max_depth: 4, n_estimators: 150 ===
Eğitim Süresi (ortalama): 6.4487 saniye
Test Süresi (ortalama): 0.0041 saniye
Accuracy (macro): 0.5029
Precision (macro): 0.5085
Recall (macro): 0.4632
F1-Score (macro): 0.4694

=== XGBoost Model 4 | learning_rate: 0.2, max_depth: 3, n_estimators: 100 ===
Eğitim Süresi (ortalama): 2.9463 saniye
Test Süresi (ortalama): 0.0024 saniye
Accuracy (macro): 0.5162
Precision (macro): 0.5140
Recall (macro): 0.4821
